In [66]:
import numpy as np

1. Construir un clase  que permita definir una red neuronal con la topología
deseada y la función de activación para cada capa, para ello deberá construir una funcion Topology con el número de capas de la red neuronal.

Se define la función de activación:

In [67]:
def act_function(x, activation):
  if activation == "sigmoid":
    f = 1 / (1 + np.exp(-x))
    fp = f * (1 - f)
    return f, fp

  elif activation == "tanh":
    f = np.tanh(x)
    fp = 1 - f**2
    return f, fp

  elif activation == "relu":
    f = np.maximum(0, x)
    fp = (x > 0).astype(float) # la derivada es 1 para x > 0, sino 0
    return f, fp

  elif activation is None or activation == "None":
    # si no se especifíca la función de activación se usa la función identidad
    return x, np.ones_like(x)

  else:
    raise ValueError(f"Activation function '{activation}' not supported.")

Se define la clase para una capa individual de la red:

In [68]:
class layer_nn():
  def __init__(self, act_fun_name, nlayer_present, nlayer_before):
    # se inicializan los parámetros de forma aleatoria entre -1 y 1
    self.theta = 2 * np.random.random((nlayer_present, nlayer_before)) - 1
    self.B = 2 * np.random.random((nlayer_present, 1)) - 1
    self.act_fun_name = act_fun_name

  # A_prev es el output de la anterior capa
  def output(self, A_prev):
    self.Z = np.dot(self.theta, A_prev) + self.B
    self.A, _ = act_function(self.Z, self.act_fun_name)
    return self.A, self.Z

2. Construir un generalizacion de la red, en el que entrada el valor inicial
y la red neuronal completa arroje la salida y la actualizacion de la red con los parametros deseados:

```python
A, nn = forward_pass(A0, nn_red)
```

Se implementa la clase `NeuralNetwork`

In [69]:
class NeuralNetwork():
  def __init__(self, topology, activations):
    self.layers = []
    for i in range(1, len(topology)):
      nlayer_present = topology[i]
      nlayer_before = topology[i-1]
      act_fun_name = activations[i]
      self.layers.append(layer_nn(act_fun_name, nlayer_present, nlayer_before))

  def forward_propagation(self, A0):
    A_current = A0
    for layer in self.layers:
      A_current, _ = layer.output(A_current)
    return A_current

Se define la función `forward_pass`:

In [70]:
def forward_pass(A0, nn_red):
  final_A = nn_red.forward_propagation(A0)
  return final_A, nn_red

3. Encontrar la funcion de coste.


$$-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)) \tag{7}$$

Se define la función `compute_cost(A, Y)` que calcula la función de coste dada la salida predicha `A` y las etiquetas `Y`:

In [71]:
def compute_cost(A, Y):
  m = Y.shape[1]  # número de ejemplos de entrenamiento

  # Calcula el coste
  cost = -(1 / m) * np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A))

  return cost

4. Construir un codigo que permita realizar el BackwardPropagation.

Se añade el método `backward_propagation` a la clase `NeuralNetwork` que calcula gradientes y actualiza los pesos y bias en todas las capas:

In [72]:
class NeuralNetwork():
  def __init__(self, topology, activations):
    self.layers = []
    self.activations = []
    for i in range(1, len(topology)):
      nlayer_present = topology[i]
      nlayer_before = topology[i-1]
      act_fun_name = activations[i]
      self.layers.append(layer_nn(act_fun_name, nlayer_present, nlayer_before))

  def forward_propagation(self, A0):
    self.activations = [A0]
    A_current = A0
    for layer in self.layers:
      A_current, Z_current = layer.output(A_current)
      self.activations.append(A_current)
    return A_current

  def backward_propagation(self, X, Y, learning_rate):
    final_A = self.forward_propagation(X)

    m = X.shape[1]

    # calculate dAL para la última capa
    dAL = -(np.divide(Y, self.activations[-1]) - np.divide(1 - Y, 1 - self.activations[-1]))

    current_dA = dAL

    # se realiza el backward por las capas
    # len(self.layers) es L (número de capas excluyendo el input)
    # self.layer[l_idx] corresponde a la capa l_idx + 1
    # self.activations[l_idx] corresponde a A^[l_idx]
    for l_idx in reversed(range(len(self.layers))):
      layer = self.layers[l_idx]

      Z_l = layer.Z
      A_prev_l = self.activations[l_idx] # A^[l] (activación desde la anterior capa)

      # obtiene la derivada de la función de activación para la capa actual
      _, fp = act_function(Z_l, layer.act_fun_name)

      dZ_l = current_dA * fp

      # calcula los gradientes para los pesos (dTheta) y los bias (dB)
      dTheta_l = (1 / m) * np.dot(dZ_l, A_prev_l.T)
      dB_l = (1 / m) * np.sum(dZ_l, axis=1, keepdims=True)

      # actualiza los pesos y los bias
      layer.theta -= learning_rate * dTheta_l
      layer.B -= learning_rate * dB_l

      # calcula dA para las anteriores capas para continuar el backpropagation
      current_dA = np.dot(layer.theta.T, dZ_l)

# Probando la generalización de la NN

In [73]:
topology = [2, 3, 1]
activations = [None, 'relu', 'sigmoid']

Se crea una muesta de inputs X (2 características, 4 ejemplos):

In [74]:
X = np.array([[0.6, 0.9, 0.7, 0.95],
              [0.3, 0.6, 0.68, 0.09]])

Se crean las etiquetas Y (1 output, 4 ejemplos):

In [75]:
Y = np.array([[0, 1, 1, 0]])

Se inicializa la red neuronal:

In [76]:
nn = NeuralNetwork(topology, activations)

In [77]:
learning_rate = 0.5
epochs = 1000

In [78]:
for epoch in range(epochs):
  # forward propagation
  A_final = nn.forward_propagation(X)

  cost = compute_cost(A_final, Y)

  # imprime el costo cada 100 épocas
  if epoch % 100 == 0:
    print(f"Época {epoch}, Coste: {cost:.4f}")

  # backward propagation
  nn.backward_propagation(X, Y, learning_rate)

Época 0, Coste: 0.7748
Época 100, Coste: 0.1062
Época 200, Coste: 0.0196
Época 300, Coste: 0.0089
Época 400, Coste: 0.0055
Época 500, Coste: 0.0038
Época 600, Coste: 0.0029
Época 700, Coste: 0.0023
Época 800, Coste: 0.0019
Época 900, Coste: 0.0016


In [79]:
print(f"Costo final después de {epochs} épocas: {cost}")

Costo final después de 1000 épocas: 0.001412394722058705


Se muestra la predicción final:

In [80]:
final_predictions = nn.forward_propagation(X)
print("predicción final (A_final):\n", final_predictions)
print("\nEtiquetas (Y):\n", Y)

predicción final (A_final):
 [[2.99991419e-03 9.97695486e-01 9.99677802e-01 8.07066507e-06]]

Etiquetas (Y):
 [[0 1 1 0]]
